In [9]:
import folium
import branca.colormap as cm
import clustering as cl
import networkx as nx
import spatial as sp

In [2]:
metro = nx.read_gml("data/graph/metro.gml")
metro_nodes = list(metro.nodes(data = True))
stops = metro_nodes# + bus_nodes + tram_nodes
regions = cl.cluster_stops(stops, 1)

In [3]:
s_points = [[float(i) for i in stop[1]["coord"].split(",")] for stop in stops]

In [4]:
points = [x["coord"] for _, x in regions.items()]
region_neighbors = cl.get_neighbor_list(points, radius=4.5)


In [5]:
# Weird vertices in voronoi_finite, might not work correctly
sp_regions, sp_vertices = sp.voronoi_finite(points)

# Missing: drawing voronoi over map

In [6]:
neighbor_graph = nx.Graph(region_neighbors)

In [62]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

for n in neighbor_graph:
    neighbor_graph.nodes[n]["coord"] = regions[n]["coord"]
    neighbor_graph.nodes[n]["stops"] = [member[1]["name"] for member in regions[n]["members"]]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["coord"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(neighbor_graph.nodes[n]["stops"]),
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)
    
for e in neighbor_graph.to_directed().edges:
    edge_coords = [neighbor_graph.nodes[e[0]]["coord"], neighbor_graph.nodes[e[1]]["coord"]]
    folium.PolyLine(edge_coords, color = "blue", weight = 1.3).add_to(m)
    
linear = cm.linear.Set1_09.scale(0, len(regions)).to_step(len(regions))
linear.caption = "Regions"
m.add_child(linear)
for key, val in regions.items():
    for elem in val["members"]:
        folium.Circle(
            radius = 100,
            location = [float(i) for i in elem[1]["coord"].split(",")],
            popup = "Region: " + str(key) + "<br>" + elem[1]["name"],
            color = linear(key),
            fill = True,
            fill_color = linear(key),
            fill_opacity=0.3,
        ).add_to(m)
    
m